이전에 간단한 대수적 구조인 모노이드를 소개했다. 모노이드는 완전히 추상적이고 순수하게 대쇼줙인 인터페이스의 사례였다. 같은 법칙을 만족한다는 공통점을 가진 일단의 연산들만으로 유용한 인터페이스를 정의할 수 있다는 새로운 사고 방식을 배울 수 있었다.

이전에 만든 라이브러리들에 존재하는 중복 코드를 추출하는 문제를 풀어 본다. 이 과정에서 새루운 추상 인터페이스 Functor와 Monad를 발견하게 될 것이다. 코드에서 이런 종류의 추상 구조를 발견하는 경험을 더욱 많이 쌍을 것임다.

functor(함수자)와 monad(모나드)는 수학의 한 분야인 범주론에서 비롯된 용어로 수학에서는 함자라고도 하며 c++에서 functor하곤 다르다.

# 11.1 함수자: map 함수의 일반화

이전장에서 다룬 라이브러리에서 적은 수의 기본 수단들을 작성하고 오직 그 기본 수단드란 이용해서 여러 조합기를 작성해 본다. 서로 다른 라이브러리의 그런 조합기들 사이에서 유사성을 발견할 수 이썼다. 예로 우리는 인수 하나를 받는 함수를 다른 어떤 자료 형식의 "문맥으로 끌어 올리는" map함수를 자료 형식마다 구현했다. Gen형식과 parser형식, Option형식에 대한 이 map함수의 서명들이다.

In [ ]:
def map[A,B](ga: Gen[A])(f: A=>B): Gen[B]
def map[A,B](pa: Parser[A])(f: A=>B): Parser[B]
def map[A,B](oa: Option[A])(f: A=>B): Option[B]

자료 구조는 다르지만 map을 구현하는 자료 형식이라는 개념을 대표하는 스칼라 형질/특질(trait)로 표현 가능

In [1]:
trait Functor[F[_]] {
    def map[A,B](fa: F[A])(f: A=>B): F[B]
}

defined trait Functor

이 특질은 map을 형식 생성자 F[\_]로 매개변수화한다. 10장에서 Foldable에 대해 했던 것과 비슷하다. 여기서 type constructor는 형식에 적용해서 형식을 생성하는 수단임을 기억하자. 예로 List는 형식 생성자이지 형식이 아니다. List 형식의 값은 존재하지 않는다. List를 Int에 적용해서 List[Int] 형식을 산출한다. Parser를 String에 적용해서 Parser[String]을 산출한다. F[\_]를 Gen, Parser 같은 특정한 형식으로 고정하는 대신, 이 Functor 특질은 F를 하나의 형식 매개변수로 두어서 프로그래머가 임의로 지정할 수 있게 한다. 다음은 List를 위한 Functor다.

In [2]:
val listFunctor = new Functor[List] {
    def map[A,B](as: List[A])(f: A=>B): List[B] = as map f
}

listFunctor: AnyRef with Functor[List] = cmd1$$user$$anonfun$1$$anon$1@d67774a

List(Option, F) 같은 형식 생성자를 가리켜 함수자(functor)라 한다. Functor[F]인스턴스는 F가 실제로 하나의 함수자임을 증명하는 증거가 된다. 단지 **인터페이스의 연산들을 순수하게 대수적인 방식으로 가지고 노는 것만으로도 유용한 함수를 발견 가능**

예, F가 하나의 함수이고 F[(A,B)]가 주어졌을 때, F를 그 쌍(pair)에 "분배해서(distribute)" (F[A], F[B])를 얻을 수 있다.

In [3]:
trait Functor[F[_]] {
    def distribute[A,B](fab: F[(A,B)]): (F[A],F[B]) = 
        (map(fab)(_._1), map(fab)(_._2))
}

: 

이 함수는 단지 형식들을 따라 작성한 것일 뿐, List/Gen/Parser/Option 같은 구체적인 자료 형식에 대해 어떤 의미인지 생각해 보자. List[(A,B)]에 distribute를 적용하면 길이가 같은 목록 2개가 산출, 하나는 A들로만 이루어짐, 다른 하나는 B로만 이루어진 목록. 이런 연산을 지퍼열기(unzip)라고 한다.

곱/product에 이런 연산이 가능하다면 쌍대곱/coproduct에 대해서도 이와 반대되는 연산을 만들 수 있을 것임

In [ ]:
def codistribute[A,B](e: Either[F[A],F[B]]): F[Either[A,B]] =
    e match {
        case Left(fa) => map(fa)(Left(_))
        case Right(fb) => map(fb)(Right(_))
    }

Gen에 codistribute는 무슨 의미일까? 우측이든 좌측이든 주어진 쪽에 대해서 Gen을 수행한다는 의미가 된다.

## 11.1.1 함수자의 법칙들

Functor같은 추상을 만들 때에는 어떤 추상 메소드들이 필요한지 고민할 뿐만 아니라 구현들이 지켜야 할 법칙(raw)들도 고민해야 한다. 하나의 추상을 위한 법칙들은 전적으로 프로그래머가 결정하는 것일 뿐 스칼라가 강제하지는 않는다 하지만 법칙은 중요하다. 이유는 아래에 있다.

* 법칙은 인터페이스의 의미론을 해당 대수를 인스턴스들과는 독립적으로 추론할 수 있을 정도의 새루운 수준으로 끌어올리는 데 도움이 된다. 예로 Monoid[A]와 Monoid[B]의 곱으로 Monoid[(A,B)]를 만든다고 할 때, 모노이드 법칙 덕분에 이 "융합된" 모노이드는 연산 역시 결합법칙을 만족한다는 결론을 내릴 수 있다.
* Functor 같은 추상 인터페이스의 함수들로부터 조합기들을 파생할 때 법칙들에 의존하는 경우가 많다. 나중에 이에 대한 예들을 보게 될 것이다.

Functor에 대해서는 7장에서 Par자료 형식에 대해 처음 소개했던 친숙한 법칙을 명시하기로 하자.

In [ ]:
map(x)(a=>a) == x

자료 구조 x에 항등 함수를 사상하는 것 자체가 하나의 항등 함수이어야 한다는 것. map을 자료 구조를 변경해서는 안된다. 즉, Some이 None이 되는 일은 없어야 한다. **map은 오직 자료구조의 요소들만 수정해야하며 구조 자체의 형태는 그대로 두어야 한다**

In [ ]:
def distribute[A,B](fab: F[(A,B)]): (F[A], F[B])
def codistribute[A,B](e: Either[F[A],F[B]]): F[Either[A,B]]

F가 함자라는 사실 외에는 F에 대허서 아는 바가 없지만, 이 법칙은 반환된 값들이 그 인수들과 같은 형태임을 보장해 준다.

# 11.2 모나드: flatMap함수와 unit함수의 일반화

Functor는 라이브러리들에서 추출할 수 있는 여러 추상 중 하나일 뿐이다. 좀 더 흥미로운 Monad를 살펴보자.

In [ ]:
def map2[A,B,C](fa: Gen[A], fb: Gen[B])(f:(A,B)=>C): Gen[C] =
    fa flatMap(a => fb map(b => f(a,b)))
def map2[A,B,C](fa: Parser[A], fa: Parser[B])(f:(A,B)=>c): Parser[C] =
    fa flatMap(a => fb map(b => f(a,b)))
def map2[A,B,C](fa: Option[A], fa: Option[B])(f:(A,B)=>c): Option[C] =
    fa flatMap(a => fb map(b => f(a,b)))

이 함수들의 공통점이 없어 보이는 서로 다른 자료 휴여식이 작용하자민 구현은 모두 동일하다. 다른 점은 함수가 적용되는 구체적인 자료 형식뿐. 이들은 좀더 일반적인 패턴의 사례들이라는 것이고 그렇다면 중복을 줄일 수 있다. 

## 11.2.1 Monad 특질

Parser/Gen/Par/Option등 지금까지 살펴본 자료 형식을 하나로 묶는 공통점은 이들이 Monad라는 사실이다. Functor나 Foldable에 했던 것처럼 map2와 기타 여러 함수를 구체적인 자료 형식마다 중복해서 정의하는 대신 모나드를 대표하는 스칼라 특질에서 한 번만 정의해 두고 재사용하는 것이 바람직

In [ ]:
trait Mon[F[_]] {
    def map2[A,B,C](fa: F[A], fb: F[B])(f: A=>B): F[C] =
        fa flatMap (a => fb map (b => f(a,b)))
}

이 구현은 Parser/Gen/Option을 Mon[F]에서 F로 대치했을 뿐임.

In [ ]:
trait Mon[F[_]] {
    def map[A,B](fa: F[A])(f: A=>B): F[B]
    def flatMap[A,B](fa: F[A])(f: A=>F[B]): F[B]
    
    def map2[A,B,C](fa: F[A], fb: F[B])(f: A=>B): F[C] =
        fa flatMap (a => fb map (b => f(a,b)))
}

이런 변경은 다소 기계적이었다. 그냥 map2의 구현을 조사해서 map2가 홏 루하는 모든 함수(map, flatMap)를 인터페이스의 적절한 추상 메소드로 추가했을 뿐임. 그럼 map2를 구성하는 기본 수단이 flatMap과 map일까? map을 unit과 flatMap으로 나눌 수 있다.

In [ ]:
def map[A,B](f: A=>B): Gen[B] = 
    flatMap(a => unit(f(a)))

따라서 최소한의 기본 수단 집합은 unit과 flatMap이어야 한다. 이제부터 할 일은 이 함수들이 정의되어 있는 모든 자료 형식을 하나의 개념으로 묶는 것이다. 그러한 개념을 대표하는 특질의 이름을 Monad라고 하자. 이 Monad 특질은 추상 함수 flatMap과 unit으로 구성되며 map과 map2의 기본 구형을 제공한다.

In [ ]:
trait Monad[F[_]] extends Functor[F] {
    def unit[A](a: => A): F[A]
    def flatMap[A,B](ma: F[A])(f: A=>F[B]): F[B]
    
    def map[A,B](ma: F[A])(f: A=>B): F[B] = 
        fa flatMap (a => unit(f(a)))
    def map2[A,B,C](fa: F[A], fb: F[B])(fab: (A,B)=>C): F[C] = 
        fa flatMap (a => fb map (b => f(a,b)))
}

> 모나드라는 용어
> 이 특질의 이름을 Monad 대신 FlatMppable이나 Unicorn, Bicycle이라고 할 수도 있다. 그러나 **모나드**는 이미 널리 쓰이는 완벽하게 좋은 이름이다. 이 용어는 수많은 함수적 프로그래밍 개념들에 영감을 준 수학의 한 분야인 범주론에서 비롯되었다. 모나드는 의도적으로 모노이디와 비슷하게 만들어진 용어이며 두 개념 사이에는 깊은 연관 관계가 존재한다. 좀 더 자세한 내용은 웹 부록의 이번 장 참고 자료를 보기 바란다.

In [ ]:
object Monad {
    val genMonad = new Monad[Gen] {
        def unit[A](a: => A): Gen[A] = Gen.unit(a)
        def flatMap[A,B](ma: Gen[A])(f: A => Gen[B]): Gen[B] =
            ma flatMap f
    }
}

unit과 flatMap을 구현하면 map과 map2가  절로 생긴다. Monad에선 map과 map2를 한번만 구현하면 된다. 그런데 **이는 시작일 뿐이고 이처럼 한번만 구현하면 되는 함수들이 꽤 있다.**

# 11.3 모나드적 조합기

모나드를 위한 기본 수단들을 갖추었으나 이전 장들에서 모나드적 자료 형식들을 식별하고 그 형식들에 대해 어떤 함수들을 구현했는지 살펴보기 바란다. 그 함수들 중에는 모든 모나드에 대해 한번만 구현하면 되는 것들이 많다.

Gen과 Parser에 대한 조합기로 listOfN이 있었다. 이 조합기는 파서 또는 생성기를 n번 되풀이해서 그 길이의 입력을 인식하고 파서 또는 그 개수 만큼의 목록들을 생성하는 생성기를 얻는데 쓰였다. 이 조합기를 Monad 특질에 추가해서 모든 모나드 F에 대해 구현할 수 있다. 이 조합기에는 listOfN대신 replicateM(replicate in a monad: 모나드의 중복) 같은 좀 더 일반적인 이름을 부여해야 마땅할 것이다.

In [ ]:
def product[A,B](ma: F[A], mb: F[B]): F[(A,B)] = map2(ma, mb)((_,_))

# 11.4 모나드 법칙

함수자 법칙들이 Monad에 대해서도 성립할 것은 확실하다. Monad[F]도 일종의 Functor[F]이기 때문이다. flatMap과 unit을 규정하는 법칙은 어떤 것일까?

## 11.4.1 결함법칙

세개의 모나드적 값을 하나로 조합한다고 할 때 셋 중 어떤 둘을 먼저 조합해야 할까? 그것이 중요할까? 이 질문에 답을 얻기 위해 추상 수준을 한 단계 낮춰서 Gen모나드를 이용한 구체적이고 간단한 예를 살펴보자.

제품 주문 시스템을 검사하는 과정에서 몇몇 모의(mock) 주문을 만들어야 하는 상황을 가정. Order라는 경우 클래스(case class)가 갖추어져 있다고 할 때, 모의 주문을 위한 주문 생성기는 다음과 같이 정의하면 될 것이다.

In [ ]:
case class Order(item: Item, quantity: Int)
case class Item(name: String, price: Double)

val getOrder: Gen[Order] = for {
    name <- Gen.striongN(3)
    price <- Gen.uniform.map(_ * 10)
    quantity <- Gen.choose(1,100)
} yield Order(Item(name, price),quantity)

In [ ]:
val genItem: Gen[Item] = for {
    name <- Gen.stringN(3)
    price <- Gen.uniform.map(_ * 10)
} yield Item(name, price)
val genOrder: Gen[Order] = for {
    item <- getItem
    quantity <- Gen.choose(1,100)
} yield Order(item, quantity)

In [ ]:
Gen.nextString(3).flatMap{ name =>
    Gen.nextDouble.flatMap{ price =>
        Gen.nextInt.map{ quantity =>
            Order(Item(name, price), quantity)
        }
    }
}

In [ ]:
Gen.nextString.flatMap { name =>
    Gen.nextDouble{ price =>
        Item(name, price)
    }.flatMap{ item =>
        Gen.nextInt.map { qunatity =>
            Order(item, quantity)
        }
    }

이런 식으로 전개해 보면 두 구현이 정확히 동일한 것이 아님이 명확해진다. 그래도 for-함축을 살펴보면 두 구현이 정화ㄱ히 동일한 일을 하리라고 가정하는 것이 완벽하게 합당해 보인다. 사실 둘이 하는 일이 다르다면 그것이 더 놀라운 일이다. 이런한 확신은 flatMap이 결합법칙을 만족한다는 가정에서 비롯된 것이다.

In [ ]:
x.flatMap(f).flatMap(g) == x.flatMap(a => f(a).flatMap(g))

이 법칙은 해당 형식의 모든 x,f,g 값에 대해 성립해야 한다. 즉, Gen 뿐만 아니라 Parser나 Option을 비롯한 모든 모나드에 성립해야 한다.

## 11.4.2 특정 모나드의 결합법칙 성립 증명

이 법칙이 Option에서 성립함을 증명해 보자 그냥 앞의 등식의 양변에서 x를 None이나 Some(v)로 치환하기만 하면 된다.

In [ ]:
None.flatMap(f).flatMap(g) == None.flatMap(a => f(a).flatMap(g))

In [ ]:
Some(v).flatMap(f).flatMap(g) == Some(v).flatMap(a => f(a).flatMap(g))
f(v).flatMap(g) == (a => f(a).flatMap(g))(v)
                 ==  f(v).flatMap(g)

### 클라이슬리 합성: 결합법칙에 관한 좀 더 명확한 관점

방금 논의한 법칙이 결합법칙이라는 점을 인ㅅ힉하기가 좀 어려울 수 있다. 모노이드에 관한 결합법칙은 다음과 같이 명확하다.

In [ ]:
op(op(x,y),z) == op(x,op(y,z))

모노이드에 관한 결합법칙은 이와는 모습이 많이 다르다. 다행히 F[A] 같은 형식의 모나드적 값이 아니라 **A=>F[B]** 같은 형식의 모나드적 함수를 고려하면 이 법칙을 좀 더 명확하게 만들 수 있다. 그런 종류의 함수를 **클라이슬리 화살표(Kleisli arrow)**라 부른다.

In [ ]:
def compose[A,B,C](f: A => F[B], g: B => F[C]): A => F[C]

## 11.4.3 항등법칙

모나드 법칙의 다른 하나인 항등법칙은 이해하기가 상당히 쉽다. 모노이드에서 append에 대한 항등원 zero가 존재하듯이, 모나드에서 compose에 대한 항등원이 존재한다. unit이 바로 그 항등원이며 애초에 unit이란 이름을 붙인 이유가 바로 그것이다.

In [ ]:
def unit[A](a: => A) = F[A]

이 함수는 적절한 형식을 compose의 인수로 전달한다. 결과적으로 임의의 모나드와 unit의 합성ㅇ느 원래의 그 모나드와 같다. 통상적으로 이는 다음과 같은 두개의 법칙, 즉 왼쪽 항등법칙과 오른쪽 항등법칙으로 표현된다.

In [ ]:
compose(f, unit) == f
compose(unit, f) == f

이 법칙들을 flatMap으로 표현할 수도 있다.

In [ ]:
flatMap(x)(unit) == f
flatMap(unit(y))(f) == f(y)

# 11.5 도대체 모나드란 무엇인가?

Monad의 인터페이스에는 뭔가 이상한 점이 있다. 모나드 인스턴스들에 주어지는 자료 형식들 사이에는 공통점이 거의 없다. Monad가 그 형식들에서 종복되는 코드를 추출한다는 점은 이해가 가지만, 그래도 모나드가 정확하게 무엇인지 아직 명확하지 않은 독자들이 있을 것이다 모나드라는 것이 과연 무엇을 의미할까?

인터페이스라는 것을 어떤 추상 자료 형식에 대한 비교적 완비된 API를 제공하는 어떤 것 단지 구체적인 표현을 추상화한 어떤 것으로 생각해 온 독자들이 있을 것이다. 사실 단일 연결 목록과 배열 기반 목록은 내부적으로는 다르게 구현되지만, 유용하고 구체적인 응용 프로그램 코드를 작성하는 관점에서 본다면 공통의 인터페이스다. Monad 조합기들은 주어진 그리고 모나드가 될 수 있는 자료 형식에 대한 전체 API중 일부만 차지한다. 즉, Monad는 한두 형식을 일반화하는 것이 아니라, Monad인터페이스와 법칙을 만족할 수 있는 아주 다양하고 많은 자료 형식을 일반화한다.

지금까지 기본 Monad 조합기들의 최소 집합은 다음 세가지였다. 그리고 Monad의 인스턴스들은 이 세 집합 중 하나의 구현들을 제공해야 한다.

* unit과 flatMap
* unit과 compose
* unit, map, join

그리고 만족해야 할 모나드 법칙은 두가지 즉 결합법칙과 항등법칙이다.

> 모나드는 모나드적 조합기들의 최소 집합 중 하나를 결합법칙과 항등법칙을 만족하도록 구현한 것이다.

이는 엄밀하고 간결하며 완벽하게 실현 가능한 정의이다 사실 이정의는 엄밀할 뿐만 아니라 유일하게 옳은 정의이다. 모나드는 전적으로 그 연산들과 법칙들로만 정의된다. 그 이상도, 그 이하도 아니다. 그러나 좀 불만스러운 점이 있다. 이 정의는 모나드의 함의가 무엇인지 즉 모나드가 무엇을 뜻하는지는 그리 잘 말해주지 않는다. 문제는 사실 이 정의가 일종의 동어반복이라는 점이다. 독자가 초보 프로그래머라고 하더라도 이제는 프로그래밍과 관련해서 상당한 지식을 갖추고 있을 텐데 이 정의는 그런 지식과는 전혀 통하지 않는다.

모나드가 무엇이고 어떻게 유용한지 진정으로 이해하기 위해서는 모나드를 독자가 이미 알고 있는 것들의 관점에서 고찰하고 그것들을 더 넓은 문맥에 연관시킬 수 있어야 한다. 모나드가 뜻하는 것이 무엇인지 어느 전도 감을 잡을 수 있도록 구체적인 모나드 두개를 살펴보면서 그 행동 방식을 비교해 보자.

## 11.5.1 항등 모나드

모나드를 그 볼질까지 파혜쳐 보자는 취지에서 가장 단순하고 흥미로운 부류인 항등 모나드(identity monad)를 살펴보자. 이 모나드의 형식은 다음과 같다.

In [11]:
case class Id[A](value: A)

defined class Id

Id는 그냥 간단한 래퍼(wrapper)이다. Id를 A에 적용하는 것은 항등 연산이다. 감싸인 형식과 감싸지 않은 형식이 완전히 동형(isomorphic)이기 때문이다, 즉 한 형식에서 다른 형식으로 갔다가 다시 원래의 형식으로 돌아와도 정보가 전혀  소실되지 않는다. 그럼 항등 모나드는 어떤 의미일까?

In [ ]:
Id("hello, ") flatMap{ a =>
    Id("monad!") flatMap{ b =>
        Id(a+b)
    }
}

In [ ]:
for {
    a <- Id("hello, ")
    b <- Id("monad!")
} yield a+b

항등 모나드에 대한 flatMap의 동작(action)은 그냥 변수 치환이다. 변수 a,b가 각각 "hello, ", "monad!"에 묶인 후 표현식 a+b로 대입된다. 같은 동작을 Id래퍼없이 그냥 스칼라 자체의 변수를 이용해서 작성할 수도 있다.

In [13]:
val a = "hello, "
val b = "monad!"

a+b

a: String = "hello, "
b: String = "monad!"
res3_2: String = "hello, monad!"

Id래퍼 말고는 차이가 없다. 이 점에서 모나드가 무슨 의미인지에 대햐나 답의 일부를 얻을 수 있다. 즉, 모나드는 변수의 도입과 결속(binding), 그리고 변수 치환 수행을 위한 문맥을 제공한다고 할 수 있다.

## 11.5.2 State 모나드와 부분 형식 적용

6장에서 State자료 형식을 논의했다. State에 대해 map과 flatMap을 비롯한 여러 조합기를 구현했음을 기억할 것이다.

In [ ]:
case class State[S, A](run: S => (A, S)) {
    def map[B](f: A => B): State[S, B] =
        State(s => {
            val (a, s1) = run(s)
            (f(a), s1)
        })
    def flatMap[B](f: A => State[S, B]): State[S, B] = 
        State(s => {
            val (a, s1) = run(s)
            f(a).run(s1)
        })
}

형태로 볼 때 State는 모나드가 최기에 충분, State의 형식 생성자는 형식 인수 두개를 받지만 Monad는 ㅇ니수가 하나인 어떤 구체적인 S에 대한 State[S, \_]는 Monad가 받아들일 수 있는 형태이다. 이는 State에 단 하나의 모나드 인스턴스가 있는 것이 아니라 여러 인스턴스들(구체적인 S마다 하나씩)의 부류(family)가 있음을 뜻한다. 형식 인수 S가 어떤 구체적인 형식으로 고정되도록 State를 부분적으로 적용할 수 있다면 좋을 것이다.

이는 함수를 부분적으로 적용하는 것과 상당히 비슷하다. 단지 형식 수준에서 적용한다는 점이 다를 뿐이다. 예를 들어 다음과 같은 IntState 형식 생성자를 만들 수 있다. 이것은 첫 형식 인수가 Int로 고정된 State의 별칭이다.

In [ ]:
type IntState[A] = State[Int, A]

In [ ]:
object IntStateMonad extends Monad[IntState] {
    def unit[A](a: => A): IntState[A] = State(s => (a, s))
    def flatMap[A](st: IntState[A])(f: A => IntState[B]): IntState[B] =
        st flatMap f
}

물론 구체적인 상태 형식마다 이렇게 개별적인 Monad 인스턴스를 작성해야 한다면 코드가 엄청나게 중복될 것이다. 안타깝게도 익명 함수를 만들 때와는 달리 스칼라에서 밑줄 구문을 이용해서 그냥 State[Int, \_]라고 표기한다고 해서 저적로 익명 형식 생성자가 만들어지지는 않는다. 대신, 형식 수준에서 람다 구문과 비슷한 어떤 것을 사용할 수 있는 예를 들어 IntState를 다음과 같이 즉석에서(inline) 선언하는 것이 가능하다.

In [ ]:
object IntStateMonad extends
    Monad[({type IntState[A] = State[Int, A]})#IntState]

괄호 바깥에서는 # 구문을 이용해서 그 IntState 멤버에 접근한다. 값 수준에서 객체의 멤버에 접근할 때 마침표(.)를 사용하는 것처럼 형식 수준에서 형식 멤버에 접근할 때에는 # 기호를 사용한다. 즉석에서 선언된 형식 생성자를 스칼라에서는 형식 람다(type lambda)라고 부르기도 한다.

In [ ]:
def stateMonad[S] = new Monda[({type f[x] = State[S,x]})#f] {
    def unit[A](a: => A): State[S, A] = State(s => (a, s))
    def flatMap[A,B](st: State[S,A])(f: A=> State[S,B]): State[S,B] =
        st flatMap f
}

여기서 잠깐, Id모나드와 State모나드의 차이점을 살펴보자. 이전에 논의했듯이 State에 대한 기본 수단 연산(모나드적 연산 unit과 flatMap 이외의)으로는 현재 상태를 읽는 getState와 새 상태를 설정하는 setState가 있다.

In [ ]:
def getState[S]: State[S, S]
def setState[S](s: => S): State[S, Unit]

이 조합기들이 State에 대한 기본 수단들의 최소 집합을 형성한다는 점도 기억할 것이다. 이들과 모나드적 기본 수단들(unit, flatMap)은 State 자료 형식으로 할 수 있는 모든 것을 완전하게 명시한다.

for 블록 안에서 getState와 setState가 쓰이는 방식에 주목하기 바란다.

In [ ]:
val F = stagteMonad[Int]

def zipWithIndex[A](as: List[A]): List[Int, A] =
    as.foldLeft(F.unit(List[(Int, A)]()))((acc,a) => for{
        xs <- acc
        n <- getState
        _ <- setState(n+1)
    } yield(n,a)::xs).run(0)._1.reverse

Id의 동작과 State의 동작 사이의 이러한 차이점에서 모나드의 일반적인 의미에 관해 어떤 것을 배울 수 있을까? flatMap 호출들의 연쇄는 변수에 값을 배정하는 명령문들로 이루어진 명령식 프로그램과 비슷하며 모나드는 각 명령문의 경계에서 어떤일이 일어나는지를 명시한다.

# 11.6 요약

이 책에서 자주 등장하는 패턴을 모나드라는 단일 개념으로 통합했다. 모나드를 이용하면 언뜻 보기에는 **공통점이 전혀 없는 서로 다른 자료 형식들에 대한 여러 조밥기를 단 한번만 작성**할 수 있다.

이런 추상적인 주제를 단번에 완전히 이해할 수는 없지만 서로 다른 관점에서 주제를 다시 고찰하고 반복적인 접근 방식이 필요하다. 독자가 **새로운 모나드나 모나드의 새로운 용버을 발견한다면 또는 새로운 문맥에서 모나드가 등장하는 것을 보게 된다면, 모나드에 대한 새로운 통찰을 얻게 될 것**이다. 그리고 **그런 일이 일어날 때마다 이런 생각을 하게 될 것이다. "전에도 모나드가 뭔지 이해했다고 생각했지만, 이제는 정말로 이해한 것 같아.**

# Thank you so much, Q&A